# Data Preprocessing - Parte 4 (Maps)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def load_data(file_path, sep=',', encoding='utf-8'):
    """Load data from a CSV file into a pandas DataFrame."""
    try:
        data = pd.read_csv(file_path, sep=sep, encoding=encoding)
        print("Data loaded successfully.")
        return data
    except Exception as e:
        print(f"An error occurred while loading the data: {e}")
        return None

def summarize_data(data):
    """Generate summary statistics of the DataFrame."""
    if data is not None:
        summary = data.describe()
        print("Data summary:")
        print(summary)
        return summary
    else:
        print("No data to summarize.")
        return None

In [3]:
data = load_data("../data/apac_data.csv", sep=';')
summary = summarize_data(data)

Data loaded successfully.
Data summary:
              EVENTS     FATALITIES  POPULATION_EXPOSURE             ID
count  197730.000000  197730.000000         9.976600e+04  197730.000000
mean        3.459632       2.036479         9.523680e+04    1538.280291
std         7.029385       9.795868         2.026675e+05     993.437542
min         1.000000       0.000000         1.000000e+00       1.000000
25%         1.000000       0.000000         1.054400e+04     593.000000
50%         1.000000       0.000000         3.482700e+04    1314.000000
75%         3.000000       0.000000         9.525400e+04    2283.000000
max       190.000000     618.000000         5.030356e+06    4157.000000


In [4]:
data.head()

,WEEK,REGION,COUNTRY,ADMIN1,EVENT_TYPE,SUB_EVENT_TYPE,EVENTS,FATALITIES,POPULATION_EXPOSURE,DISORDER_TYPE,ID,CENTROID_LATITUDE,CENTROID_LONGITUDE
0,31-dicembre-2016,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,6,15,NaN,Political violence,1,"36,966","73,3417"
1,07-gennaio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,5,28,NaN,Political violence,1,"36,966","73,3417"
2,21-gennaio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,1,4,NaN,Political violence,1,"36,966","73,3417"
3,04-febbraio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,1,4,NaN,Political violence,1,"36,966","73,3417"
4,11-febbraio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,4,41,NaN,Political violence,1,"36,966","73,3417"


In [5]:
# Filtering for Afghanistan
data['YEAR'] = data['WEEK'].apply(lambda x: int(x.split('-')[2]))

afg_data = data[data['COUNTRY'] == 'Afghanistan']

# Grouping by Admin1, Event_type and summing Events
grouped_data = afg_data.groupby(['ADMIN1', 'EVENT_TYPE']).agg({'EVENTS': 'sum'}).reset_index()

grouped_data['ADMIN1'].value_counts()

ADMIN1
Badakhshan    6
Badghis       6
Wardak        6
Urozgan       6
Takhar        6
Samangan      6
Parwan        6
Panjshir      6
Paktika       6
Paktia        6
Nuristan      6
Nimruz        6
Nangarhar     6
Logar         6
Laghman       6
Kunduz        6
Kunar         6
Khost         6
Kapisa        6
Baghlan       6
Balkh         6
Bamyan        6
Daykundi      6
Farah         6
Faryab        6
Ghazni        6
Ghor          6
Helmand       6
Herat         6
Jowzjan       6
Kabul         6
Kandahar      6
Zabul         6
Sar-e Pol     5
Name: count, dtype: int64

In [6]:
grouped_data.head()

,ADMIN1,EVENT_TYPE,EVENTS
0,Badakhshan,Battles,842
1,Badakhshan,Explosions/Remote violence,233
2,Badakhshan,Protests,49
3,Badakhshan,Riots,15
4,Badakhshan,Strategic developments,74


In [7]:
# For each ADMIN1, find the EVENT_TYPE with the maximum EVENTS
max_events = grouped_data.loc[grouped_data.groupby('ADMIN1')['EVENTS'].idxmax()]

max_events.head()

max_event = max_events['EVENTS'].max()
print(f"Maximum number of events in a single ADMIN1: {max_event}")

min_event = 0

Maximum number of events in a single ADMIN1: 4119


In [8]:
# Adding a new entry for ADMIN1 == 'Sar-e Pol' and EVENT_TYPE == 'Riots' and EVENTS == 0
if not ((grouped_data['ADMIN1'] == 'Sar-e Pol') & (grouped_data['EVENT_TYPE'] == 'Riots')).any():
    new_entry = pd.DataFrame({
        'ADMIN1': ['Sar-e Pol'],
        'EVENT_TYPE': ['Riots'],
        'EVENTS': [0],
        'NORMALIZED_EVENTS': [0.0]
    })
    grouped_data = pd.concat([grouped_data, new_entry], ignore_index=True)


In [9]:
# Normalizing the EVENTS for color scaling
grouped_data['NORMALIZED_EVENTS'] = (grouped_data['EVENTS'] - min_event) / (max_event - min_event)
grouped_data.head()

,ADMIN1,EVENT_TYPE,EVENTS,NORMALIZED_EVENTS
0,Badakhshan,Battles,842,0.204419
1,Badakhshan,Explosions/Remote violence,233,0.056567
2,Badakhshan,Protests,49,0.011896
3,Badakhshan,Riots,15,0.003642
4,Badakhshan,Strategic developments,74,0.017966


In [10]:

print(grouped_data[grouped_data['ADMIN1'] == 'Sar-e Pol'])

        ADMIN1                  EVENT_TYPE  EVENTS  NORMALIZED_EVENTS
174  Sar-e Pol                     Battles     392           0.095169
175  Sar-e Pol  Explosions/Remote violence     120           0.029133
176  Sar-e Pol                    Protests       8           0.001942
177  Sar-e Pol      Strategic developments      48           0.011653
178  Sar-e Pol  Violence against civilians      65           0.015781
203  Sar-e Pol                       Riots       0           0.000000


In [12]:
afg_data['ADMIN1'].unique()

array(['Badakhshan', 'Badghis', 'Baghlan', 'Balkh', 'Bamyan', 'Daykundi',
       'Farah', 'Faryab', 'Ghazni', 'Ghor', 'Helmand', 'Herat', 'Jowzjan',
       'Kabul', 'Kandahar', 'Kapisa', 'Khost', 'Kunar', 'Kunduz',
       'Laghman', 'Logar', 'Nangarhar', 'Nimruz', 'Nuristan', 'Paktia',
       'Paktika', 'Panjshir', 'Parwan', 'Samangan', 'Sar-e Pol', 'Takhar',
       'Urozgan', 'Wardak', 'Zabul'], dtype=object)

In [11]:
# Grouping by event_type and summing the events
#event_summary = afg_data.groupby('EVENT_TYPE').agg({'EVENTS': 'sum'}).reset_index()
#print(event_summary)

In [26]:
#print(max_events)